In [1]:
!pip install -qU langchain sentence_transformers
!pip install -qU langchain-huggingface langchain-community
!pip install -qU faiss-cpu
!pip install ragas==0.2.0 langchain openai groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 766.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/11

In [2]:
import pandas as pd
from ragas.metrics import Faithfulness, AnswerRelevancy, LLMContextPrecisionWithoutReference
from google.colab import drive, files
import nest_asyncio
nest_asyncio.apply()
import ast

drive.mount('/content/drive', force_remount=True)

/usr/local/lib/python3.10/dist-packages/ragas/prompt/base.py:9: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.llms.prompt import PromptValue


Mounted at /content/drive


In [32]:
csv_path = '/content/drive/MyDrive/IA024/ProjetoFinal/analise_qualitativa/teste_total_gpt_sem acórdão_com acórdão.xlsx'

df = pd.read_excel(csv_path)
df['answer_cleaned'] = df['answer_cleaned'].str.replace(r"[\[\]',]", '', regex=True).str.strip()
df = df.drop(columns=['question_number'])
df = df.rename(columns={
    'question_text': 'user_input',
    'contexto': 'retrieved_contexts',
    'resposta_rag': 'response',
    'answer_cleaned': 'reference'
})
df['retrieved_contexts'] = df['retrieved_contexts'].apply(ast.literal_eval)
df.head()

'''csv_path = '/content/drive/MyDrive/IA024/ProjetoFinal/analise_qualitativa/teste_total_gpt_sem acórdão_com acórdão.xlsx'

df = pd.read_excel(csv_path)
df = df[['question_text', 'answer_cleaned', 'resposta_rag com acórdao', 'contexto com acórdao']]
df = df.rename(columns={
    'question_text': 'user_input',
    'contexto com acórdao': 'retrieved_contexts',
    'resposta_rag com acórdao': 'response',
    'answer_cleaned': 'reference'
})
df['retrieved_contexts'] = df['retrieved_contexts'].apply(ast.literal_eval)
df.head()'''

,user_input,reference,response,retrieved_contexts
0,Qual o tratamento tributário dos rendimentos d...,['As quantias recebidas por pessoa física pela...,Os rendimentos de aluguel de imóvel localizado...,[RIR2018: Art. 781. Compete ao procurador a re...
1,Qual é o tratamento tributário dos rendimentos...,"['Preliminarmente, deve-se verificar se há aco...",Os rendimentos de aluguel de imóvel localizado...,[RIR2018: Art. 781. Compete ao procurador a re...
2,Quando ocorre o fato gerador no caso de o alug...,['O fato gerador ocorre no mês em que o locatá...,O fato gerador do imposto sobre a renda em rel...,[Instrução Normativa RFB nº 1.500: Seção VI\nD...
3,O pai deu ao filho o usufruto de rendimentos d...,['Se o usufruto constar de escritura pública a...,Os rendimentos de aluguel de imóvel recebidos ...,[Decreto nº 3.000: Art. 8º Os cônjuges poderã...
4,Como tratar os rendimentos produzidos por imóv...,['Esses rendimentos são tributáveis em nome de...,Os rendimentos provenientes de um imóvel cujo ...,[RIR2018: § 4º Na apuração do ganho de capital...


In [33]:
import pandas as pd
from ragas.metrics import Faithfulness, ResponseRelevancy, LLMContextPrecisionWithoutReference
from ragas import SingleTurnSample, EvaluationDataset
import asyncio
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.metrics import  Faithfulness, SemanticSimilarity, LLMContextRecall, AnswerRelevancy
from ragas import evaluate

converted_samples=[]

for index, row in df.iterrows():
    converted_sample = SingleTurnSample(user_input=row["user_input"],
                                        response=row["response"],
                                        reference=row["reference"],
                                        retrieved_contexts=ast.literal_eval((str(row["retrieved_contexts"]))))

    converted_samples.append(converted_sample)

eval_dataset = EvaluationDataset(samples=converted_samples)

In [34]:
import os

os.environ["OPENAI_API_KEY"] = "" # Colocartchave openai
client = ChatOpenAI(model="gpt-4o-mini")
evaluator_llm = LangchainLLMWrapper(client)

In [35]:
metrics = [LLMContextRecall(), SemanticSimilarity(), Faithfulness(), AnswerRelevancy()]
results = evaluate(dataset=eval_dataset, metrics=metrics, llm=evaluator_llm)
df_result = results.to_pandas()
df_result

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_recall,semantic_similarity,faithfulness,answer_relevancy
0,Qual o tratamento tributário dos rendimentos d...,[RIR2018: Art. 781. Compete ao procurador a re...,Os rendimentos de aluguel de imóvel localizado...,['As quantias recebidas por pessoa física pela...,0.371429,0.930383,0.214286,0.962663
1,Qual é o tratamento tributário dos rendimentos...,[RIR2018: Art. 781. Compete ao procurador a re...,Os rendimentos de aluguel de imóvel localizado...,"['Preliminarmente, deve-se verificar se há aco...",0.600000,0.921873,0.500000,0.954251
2,Quando ocorre o fato gerador no caso de o alug...,[Instrução Normativa RFB nº 1.500: Seção VI\nD...,O fato gerador do imposto sobre a renda em rel...,['O fato gerador ocorre no mês em que o locatá...,1.000000,0.896978,1.000000,0.935445
3,O pai deu ao filho o usufruto de rendimentos d...,[Decreto nº 3.000: Art. 8º Os cônjuges poderã...,Os rendimentos de aluguel de imóvel recebidos ...,['Se o usufruto constar de escritura pública a...,0.600000,0.935461,0.555556,0.943490
4,Como tratar os rendimentos produzidos por imóv...,[RIR2018: § 4º Na apuração do ganho de capital...,Os rendimentos provenientes de um imóvel cujo ...,['Esses rendimentos são tributáveis em nome de...,0.666667,0.928948,0.500000,0.967558
5,Como deve ser tributada a quantia recebida por...,[Instrução Normativa RFB nº 1.500: Seção VI\nD...,A tributação da quantia recebida por locação d...,['As quantias recebidas por pessoa física pela...,0.666667,0.932418,0.583333,0.964133
6,Como apurar o rendimento tributável de aluguel...,[Instrução Normativa RFB nº 1.500: Seção VI\nD...,Para apurar o rendimento tributável de aluguel...,['Tributa-se o valor recebido de aluguel subtr...,0.888889,0.914923,1.000000,0.937306
7,Os rendimentos oriundos de contrato de arrenda...,[13370721838202026_7150516: terceiros. \nO con...,Os rendimentos oriundos de contrato de arrenda...,['Os rendimentos provenientes de arrendamento ...,1.000000,0.930129,0.750000,0.947685
8,Qual é o tratamento tributário de rendimentos ...,[Instrução Normativa RFB nº 1.500: Art. 32. Co...,Os rendimentos de aluguéis depositados em juíz...,['No caso de o locatário efetuar em juízo o de...,0.750000,0.912317,0.636364,0.954682
9,As luvas e gratificações pagas ao locador são ...,[Instrução Normativa RFB nº 1.500: Art. 32. Co...,As luvas e gratificações pagas ao locador são ...,"['Sim. As luvas, prêmios, gratificações ou qua...",0.800000,0.936115,0.857143,0.830172


In [36]:
mean_metrics = df_result[['context_recall', 'semantic_similarity', 'faithfulness', 'answer_relevancy']].mean()
median_metrics = df_result[['context_recall', 'semantic_similarity', 'faithfulness', 'answer_relevancy']].median()
print("Com Acórdão:")
print("Médias das métricas:")
print(mean_metrics)
print("\nMedianas das métricas:")
print(median_metrics)

Com Acórdão:
Médias das métricas:
context_recall         0.760212
semantic_similarity    0.926181
faithfulness           0.644998
answer_relevancy       0.939729
dtype: float64

Medianas das métricas:
context_recall         0.750000
semantic_similarity    0.930383
faithfulness           0.636364
answer_relevancy       0.950841
dtype: float64


In [ ]:
df_result.to_excel("ragas_total.xlsx", index = False)
files.download('ragas_total.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
csv_path = '/content/drive/MyDrive/IA024/ProjetoFinal/df/ragas_total.xlsx'

df = pd.read_excel(csv_path)

In [38]:
df

,user_input,retrieved_contexts,response,reference,context_recall,semantic_similarity,faithfulness,answer_relevancy
0,Quem está obrigado a apresentar a Declaração d...,['Lei nº 11.438: CAPÍTULO I\nDOS INCENTIVOS AO...,A obrigação de apresentar a Declaração de Ajus...,Está obrigada a apresentar a Declaração de Aju...,1.000000,0.933509,0.000000,0.969321
1,Pessoa física desobrigada pode apresentar a De...,['Instrução Normativa RFB nº 1.704: Seção IV \...,Uma pessoa física desobrigada da apresentação ...,Sim. A pessoa física ainda que desobrigada pod...,0.500000,0.930114,0.363636,0.000000
2,Contribuinte que é titular ou sócio de empresa...,['Lei nº 11.438: CAPÍTULO I\nDOS INCENTIVOS AO...,O contribuinte que é titular ou sócio de uma e...,Não a menos que se enquadre nas hipóteses prev...,0.000000,0.904288,0.888889,0.951807
3,"Contribuinte, que participou de quadro societá...",['RIR2018: I - a pessoa jurídica resultante da...,A pergunta busca esclarecer as obrigações fisc...,Não a menos que esteja obrigado a declarar cas...,0.000000,0.895046,0.714286,0.947639
4,Contribuinte que constou como responsável pera...,['Instrução Normativa RFB nº 2.066: I - por pe...,O contribuinte que constou como responsável pe...,Esse contribuinte está obrigado a declarar cas...,0.500000,0.900863,0.875000,0.947242
...,...,...,...,...,...,...,...,...
572,Qual é o tratamento tributário dos rendimentos...,['Lei nº 7.713: Art. 44. O imposto de que trat...,Os rendimentos produzidos por conta de depósit...,Os rendimentos obtidos em conta de depósito de...,0.000000,0.907044,1.000000,0.951989
573,Qual é o tratamento tributário dos rendimentos...,['Lei nº 9.430: § 4º Tratando-se de pessoa fís...,Os rendimentos produzidos por conta de depósit...,Tratando-se de conta de depósito de poupança o...,0.000000,0.874218,0.500000,0.943685
574,As importâncias correspondentes aos juros de c...,['Lei nº 14.754: § 3º A variação cambial de de...,As importâncias correspondentes aos juros de c...,Não. Relativamente aos juros de conta de depós...,0.000000,0.932132,0.500000,0.900737
575,Qual é o tratamento tributário aplicável na aq...,['RIR2018: § 3º O disposto neste artigo aplica...,O tratamento tributário aplicável na aquisição...,A entrega pelo licitante vencedor de títulos d...,0.666667,0.925015,0.714286,0.985530


In [39]:
mean_metrics = df[['context_recall', 'semantic_similarity', 'faithfulness', 'answer_relevancy']].mean()
median_metrics = df[['context_recall', 'semantic_similarity', 'faithfulness', 'answer_relevancy']].median()
print("Total:")
print("Médias das métricas:")
print(mean_metrics)
print("\nMedianas das métricas:")
print(median_metrics)

Total:
Médias das métricas:
context_recall         0.526823
semantic_similarity    0.916025
faithfulness           0.617318
answer_relevancy       0.894038
dtype: float64

Medianas das métricas:
context_recall         0.500000
semantic_similarity    0.920657
faithfulness           0.666667
answer_relevancy       0.947639
dtype: float64
